## Class Definition

In [1]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

class Email:
    def __init__(self, sender_email, app_password, stylesheet=None):
        """
        Initialize the Email object.

        Parameters:
        ----------
        sender_email : str
            The email address from which the emails will be sent.
        app_password : str
            The app password from Google Accounts used for auth purposes.
        stylesheet : str, optional
            An optional string that holds CSS stylesheet.
            Defaults to None, meaning default stylesheet would be applied.

        Example:
        --------
        >>> email_sender = Email("your_email@example.com", "your_app_password")
        """
        
        self.sender = sender_email
        self.password = app_password
        
        # default CSS3 stylesheet
        self.stylesheet = '''
            <style>
                body {
                  font-family: helvetica;
                }

                table {
                  text-align: center;
                  border-collapse: collapse;
                }

                table td, table th {
                  padding: 5px;
                  border: 1px solid black;
                }

                table thead {
                  color: white;
                  font-weight: bold;
                  background-color: #333;
                }
            </style>
        '''
        
        # if sender provided his stylesheet then use it
        if stylesheet != None:
            self.stylesheet = stylesheet
            
        # initializing queue for internal purposes
        self.__queue__ = []
    
    def __append__(self, html):
        """
        Appends the html to the internal message queue.

        Parameters:
        ----------
        html : str
            html string to append to the internal message queue.
        """
        self.__queue__.append(html)
        
    def create_paragraph(self, msg):
        """
        Creates a HTML paragraph (<p>) from provided message.

        Parameters:
        ----------
        msg : str
            The message to convert to HTML.
        """
        
        html = '<p>{}</p>'.format(msg)
        self.__append__(html)
        
    def create_header(self, msg, header_level=1):
        """
        Creates a HTML header (<hx>) from provided message.

        Parameters:
        ----------
        msg : str
            The message to convert to HTML.
            
        header_level : int, optional
            The header level.
            Defaults to 1 for <h1>, supported values range from 1 to 6.
        """
        
        html = '<h{hl}>{msg}</h{hl}>'.format(msg=msg, hl=header_level)
        self.__append__(html)
        
    def create_table(self, df):
        """
        Creates a HTML table (<table>) from provided dataframe.

        Parameters:
        ----------
        df: pandas.DataFrame
            The dataframe to convert to HTML table.
        """
        
        html = df.to_html(index=False)
        html = html.replace('border="1" ', '')
        html = html.replace(' style="text-align: right;"', '')
        self.__append__(html)
        
    def get_message(self):
        """
        Returns the message from the internal message queue.

        Returns:
        ----------
        A message string with appropriate HTML tags.
        """
        
        body = '<body>\n'
        for msg in self.__queue__:
            body += msg + '\n'
        
        body += '</body>'
        return body
        
    def send_email(self, to, subject, body, cc=None):
        """
        Sends an email to the specified recipient(s).

        Parameters:
        ----------
        to : str or list
            The primary recipient's email address(es).
        subject : str
            The subject line of the email.
        body : str
            The body content of the email in HTML format.
        cc : str or list, optional
            An individual email addresses or a list addresses to send a carbon copy (CC) of the email.
            Defaults to None, meaning no CC recipients will be added.

        Returns:
        -------
        email.mime.multipart.MIMEMultipart

        Raises:
        ------
        smtplib.SMTPException
            If an error occurs while sending the email.

        Example:
        --------
        >>> send_email("recipient@example.com", "Hello", "This is a test email.", cc=["cc@example.com"])
        """

        # Checking whether it's a list of or an individual receiver
        if type(to) == type([]):
            to = ', '.join(to)
            
        # Checking whether it's a list of cc receivers
        if type(cc) == type([]):
            cc = ', '.join(cc)
        
        # Setting up the message
        msg = MIMEMultipart()
        msg['From'] = self.sender
        msg['To'] = to
        msg['Subject'] = subject
        
        if cc != None:
            msg['Cc'] = cc
        
        # Adding CSS3 stylesheet to the message
        body = body.replace('<html>', '').replace('</html>', '')
        body = '<html>\n<head>' + self.stylesheet + '</head>\n' + body + '\n</html>'
        msg.attach(MIMEText(body, 'html'))
        
        # Trying to send the email using gmail server
        try:
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(self.sender, self.password)
            text = msg.as_string()
            server.sendmail(self.sender, to, text)
            print('Email sent successfully!')
        except Exception as e:
            print('Error: {}'.format(e))
        finally:
            server.quit()
            
        # Returning the message in case inspection is needed
        return msg

## Validation Logic

In [5]:
import pandas as pd

from markets_analytics import redshift

query = '''
    SELECT
        "year",
        COUNT(DISTINCT sk_customer) AS customers,
        COUNT(DISTINCT sk_order) AS orders,
        ROUND(SUM(gmv_bef_cancellation), 2) AS gmv_bef_cancellation,
        ROUND(SUM(gmv_aft_return), 2) AS gmv_aft_return,
        ROUND(SUM(pcii), 2) AS pcii
    FROM sales_and_supply.customer_orders_daily
    WHERE aggregation = '365d'
    GROUP BY "year"
    ORDER BY "year"
'''

res_df = redshift.execute(query, log=False)

query = '''
    SELECT
        EXTRACT(YEAR FROM order_date) AS "year",
        COUNT(DISTINCT sk_customer) AS customers,
        COUNT(DISTINCT sk_order) AS orders,
        ROUND(SUM(gmv_bef_cancellation), 2) AS gmv_bef_cancellation,
        ROUND(SUM(gmv_aft_ret_provision), 2) AS gmv_aft_return,
        ROUND(SUM(pc_ii_lounge), 2) AS pcii
    FROM reporting.f_lounge_sales_order_position
    GROUP BY EXTRACT(YEAR FROM order_date)
    ORDER BY EXTRACT(YEAR FROM order_date)
'''

sales_df = redshift.execute(query, log=False)

df = pd.merge(res_df, sales_df, on=['year'], how='inner')
df['customers'] = df['customers_x'] - df['customers_y']
df['orders'] = df['orders_x'] - df['orders_y']
df['gmv_bef_cancellation'] = df['gmv_bef_cancellation_x'] - df['gmv_bef_cancellation_y']
df['gmv_aft_return'] = df['gmv_aft_return_x'] - df['gmv_aft_return_y']
df['pcii'] = df['pcii_x'] - df['pcii_y']

col = ['year', 'customers', 'orders', 'gmv_bef_cancellation', 'gmv_aft_return', 'pcii']
df = df[col]
df

,year,customers,orders,gmv_bef_cancellation,gmv_aft_return,pcii
0,2011,0,0,-0.01,-0.01,384.54
1,2012,0,0,-0.01,0.00,222.01
2,2013,0,0,-0.01,-0.01,245.16
3,2014,0,0,0.19,0.19,-66.51
4,2015,0,0,0.10,0.10,430.81
5,2016,0,0,-0.02,-0.01,120.31
6,2017,0,0,-0.03,-0.02,209.55
7,2018,0,0,-0.02,-0.01,-245.55
8,2019,0,0,-0.05,-0.03,-580.19
9,2020,0,0,-0.06,-0.05,-4863.26


## Class Instance(s)

In [7]:
# Initializing the class instance
sender = 'danyal.imran@zalando.de'
app_password = 'gnrx bmwr ksyv qksy' # generated from google account settings

email = Email(sender, app_password)

In [8]:
# Calling send email function
recipients = ['danyal.imran@zalando.de']
subject = 'Normal Test'

# HTML message
html = '''
<form>
    <label for="name">Name:</label>
    <input type="text" name="name"><br><br>
    <label for="sex">Sex:</label>
    <input type="radio" name="sex" id="male" value="male">
    <label for="male">Male</label>
    <input type="radio" name="sex" id="female" value="female">
    <label for="female">Female</label> <br><br>
    <label for="country">Country: </label>
    <select name="country" id="country">
        <option>Select an option</option>
        <option value="nepal">Nepal</option>
        <option value="usa">USA</option>
        <option value="australia">Australia</option>
    </select><br><br>
    <label for="message">Message:</label><br>
    <textarea name="message" id="message" cols="30" rows="4"></textarea><br><br>
    <input type="checkbox" name="newsletter" id="newsletter">
    <label for="newsletter">Subscribe?</label><br><br>
    <input type="submit" value="Submit">
</form>
'''

email.send_email(recipients, subject, html)

Email sent successfully!


In [9]:
subject = '[Validation] Customer Orders Daily'

# HTML message
html = df.to_html(index=False)
html = html.replace('<table border="1" class="dataframe">', '<table>')

email.send_email(recipients, subject, html)

Email sent successfully!


In [11]:
# Calling send email function
#recipients = ['markets-analytics@zalando-lounge.de']
recipients = ['danyal.imran@zalando.de']
subject = '[Validation] Customer Orders Daily (Extensive)'

# Creating the message
email.create_header('Validation Report', 2) # h1 - h6
email.create_paragraph('<b>Table</b>: Customer Orders Daily')
email.create_paragraph('This table computes customer orders on a daily level and is aggregated on yearly dimension')
email.create_paragraph('<b>Note</b>: Floating Point Precision has caused errors in the decimal fields like PCII, GMV, etc.')
email.create_header('Validation Table', 3)
email.create_table(df)
email.create_paragraph('<b>Best Regards</b><br>Danyal Imran<br>Markets Analytics<br>Senior Data Analyst')

# Once message is created, get the message
body = email.get_message()

msg = email.send_email(recipients, subject, body)

# Looking at the message from email.mime.multipart.MIMEMultipart
print()
print(msg)

Email sent successfully!

Content-Type: multipart/mixed; boundary="===============0485495227137694999=="
MIME-Version: 1.0
From: danyal.imran@zalando.de
To: danyal.imran@zalando.de
Subject: [Validation] Customer Orders Daily (Extensive)

--===============0485495227137694999==
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

<html>
<head>
            <style>
                body {
                  font-family: helvetica;
                }

                table {
                  text-align: center;
                  border-collapse: collapse;
                }

                table td, table th {
                  padding: 5px;
                  border: 1px solid black;
                }

                table thead {
                  color: white;
                  font-weight: bold;
                  background-color: #333;
                }
            </style>
        </head>
<body>
<h2>Validation Report</h2>
<p><b>Table</b>: Custom